In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 21.1 MB/s eta 0:00:00


In [3]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import glob
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

import optuna
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline

from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

from tqdm import tqdm
from collections import defaultdict
from itertools import combinations

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from pytorch_tabnet.tab_network import TabNetEncoder

In [5]:
seed = 810

def set_seed(sd):
    random.seed(sd)
    np.random.seed(sd)
    os.environ['PYTHONHASHSEED'] = str(sd)
    random.seed(sd)
    np.random.seed(sd)
    torch.manual_seed(sd)
    torch.cuda.manual_seed_all(sd)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(seed)

In [6]:
data_dir = '/content/drive/MyDrive/1데이콘/ETRI_휴먼이해_인공지능/ETRI_lifelog_dataset/'

In [7]:
Q1_Q3 = pd.read_csv(data_dir + 'prep_df_Q1-3.csv')
S1_S2 = pd.read_csv(data_dir + 'prep_df_S1-2.csv')
S3 = pd.read_csv(data_dir + 'prep_df_S3.csv')

pred_df_list = [Q1_Q3,S1_S2,S3]
prep_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), pred_df_list)

metrics_train = pd.read_csv(data_dir + 'ch2025_metrics_train.csv')
sample_submission = pd.read_csv(data_dir + 'ch2025_submission_sample.csv')

In [8]:
prep_df.shape

(806, 446)

# train/test 데이터셋

In [9]:
# datetime 형으로 변환
metrics_train['lifelog_date'] = pd.to_datetime(metrics_train['lifelog_date']).dt.date
sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']).dt.date
prep_df['date'] = pd.to_datetime(prep_df['date']).dt.date
# prep_df.loc[:, 'date'] = pd.to_datetime(prep_df.loc[:, 'date']).dt.date

In [10]:
# train_df
metrics_train_renamed = metrics_train.rename(columns = {'lifelog_date': 'date'})

train_df = pd.merge(metrics_train_renamed, prep_df, on=['subject_id', 'date'], how='inner')
print(train_df.shape)

(450, 453)


In [11]:
# test_df
sample_submission_renamed = sample_submission.rename(columns = {'lifelog_date': 'date'})

test_df = pd.merge(sample_submission_renamed, prep_df, on=['subject_id', 'date'], how='inner')
print(test_df.shape)

(250, 453)


In [12]:
# def clear_columns_names(df):
#   df.columns = (df.columns
#                 .str.replace(r"[^\w]", "_", regex=True)
#                 .str.replace(r'__+','_', regex=True)
#                 .str.strip('_')
#                 )
#   return df

# train_df = clear_columns_names(train_df)
# test_df = clear_columns_names(test_df)

# X/y 설정

In [13]:
x = train_df.drop(columns=['subject_id', 'sleep_date', 'date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3'])
y = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]
test_x = test_df[x.columns]

x.fillna(0, inplace=True)
y.fillna(0, inplace=True)
test_x.fillna(0, inplace=True)

In [14]:
print('train shape:', train_df.shape)
print('train X shape:', x.shape)
print('train y shape:', y.shape)
print()
print('test X shape:', test_df.shape)
print('test shape:', test_x.shape)

train shape: (450, 453)
train X shape: (450, 444)
train y shape: (450, 6)

test X shape: (250, 453)
test shape: (250, 444)


In [15]:
def find_duplicate(df):
  return df.columns[df.columns.duplicated()].tolist()

dup_x = find_duplicate(x)
dup_test_x = find_duplicate(test_x)
print('x_중복컬럼:', dup_x)
print('test_x_중복컬럼:', dup_test_x)

x_columns = set(x.columns)
test_x_columns = set(test_x.columns)

only_x = x_columns - test_x_columns
only_test_x = test_x_columns - x_columns
print('only_x:', only_x)
print('only_x_test:', only_test_x)

x_중복컬럼: []
test_x_중복컬럼: []
only_x: set()
only_x_test: set()


# 종속변수별 features

In [16]:
# features = {
#     'Q1_feat': x.columns[x.columns.str.startswith('Q1')],
#     'Q2_feat': x.columns[x.columns.str.startswith('Q2')],
#     'Q3_feat': x.columns[x.columns.str.startswith('Q2')],
#     'S1_feat': x.columns[x.columns.str.startswith('S1')],
#     'S2_feat': x.columns[x.columns.str.startswith('S1')],
#     'S3_feat': x.columns[x.columns.str.startswith('S3')]
# }

# target_to_feat = {
#     'Q1': features['Q1_feat'],
#     'Q2': features['Q2_feat'],
#     'Q3': features['Q3_feat'],
#     'S1': features['S1_feat'],
#     'S2': features['S2_feat'],
#     'S3': features['S3_feat']
# }

In [17]:
feat = {
    'Q1': 'Q1',
    'Q2': 'Q2',
    'Q3': 'Q2',
    'S1': 'S1',
    'S2': 'S1',
    'S3': 'S3'
}

features = {k: x.columns[x.columns.str.startswith(v)] for k, v in feat.items()}

# 데이터 분할

In [18]:
targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3']
target_multi = ['S1']

split_data = {}
for col in targets_binary + target_multi:
    y = train_df[col]
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=seed)
    split_data[col] = (x_train, x_val, y_train, y_val)

# smote + lgbm


In [19]:
# def check_smote_sample(split_data, col):
#     x_train, _, y_train, _ = split_data[col]

#     smote = SMOTE(sampling_strategy='minority', random_state=seed)

#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
#     for train_idx, _ in skf.split(x_train, y_train):
#         x_fold, y_fold = x_train.iloc[train_idx], y_train.iloc[train_idx]
#         x_resampled, y_resampled = smote.fit_resample(x_fold, y_fold)

#         print(f"Before SMOTE: {y_fold.value_counts().to_dict()}")
#         print(f"After SMOTE: {pd.Series(y_resampled).value_counts().to_dict()}")
#         print(f"x_resampled shape: {x_resampled.shape}")

#         break  # 한 fold만 확인

In [20]:
lgbm_best_param_dict = {}

def smote_optuna_binary(split_date):
  for col in targets_binary:
    print(f'=== target: {col} ===')
    x_train, x_val, y_train, y_val = split_data[col]


    def objective_binary(trial):
      params = {
          'learning_rate': trial.suggest_float("lr", 1e-5, 1e-1, log=True),
          'n_estimators': trial.suggest_int('n_estimators', 50, 500),
          'max_depth': trial.suggest_int('max_depth', 2, 32),
          'num_leaves': trial.suggest_int('num_leaves', 16, 64),
          'subsample': trial.suggest_float('subsample', 0.5, 1.0),
          'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
          'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
          'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
          'path_smooth' : trial.suggest_loguniform('path_smooth', 1e-8, 1e-3),
          'num_leaves' : trial.suggest_int('num_leaves', 30, 200),
          'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
          'max_bin' : trial.suggest_int('max_bin', 100, 255),
          'feature_fraction' : trial.suggest_uniform('feature_fraction', 0.5, 0.9),
          'bagging_fraction' : trial.suggest_uniform('bagging_fraction', 0.5, 0.9),
          'random_state': seed,
          'n_jobs': -1,
          'verbosity': -1
      }

      feat = features[col]
      clf_binary = LGBMClassifier(**params)
      pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority', random_state=seed), clf_binary)

      cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
      score = cross_val_score(pipeline, x_train[feat], y_train, cv=cv, scoring='f1').mean()

      trial.report(score, step=0)
      if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

      return score

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(objective_binary, n_trials=50)

    best_params = study.best_params
    best_score = study.best_value
    lgbm_best_param_dict[col] = best_params
    print(f"{col} 최적 파라미터: {best_params}")
    print(f"{col} 최고 F1 점수: {best_score:.4f}")
    print("=" * 50, "\n")

In [21]:
smote_optuna_binary(split_data)

[I 2025-06-29 04:44:34,523] A new study created in memory with name: no-name-91c2efa9-8f21-445b-9b15-5fc2a15ebd08


=== target: Q1 ===


[I 2025-06-29 04:44:39,610] Trial 0 finished with value: 0.5630972212054146 and parameters: {'lr': 1.5106249671860432e-05, 'n_estimators': 339, 'max_depth': 31, 'num_leaves': 17, 'subsample': 0.5023240118702073, 'colsample_bytree': 0.7526598267090864, 'reg_alpha': 0.6834515648539062, 'reg_lambda': 4.603537484050716, 'path_smooth': 4.1267228833656876e-08, 'min_data_in_leaf': 74, 'max_bin': 172, 'feature_fraction': 0.6605739639515698, 'bagging_fraction': 0.592452365530872}. Best is trial 0 with value: 0.5630972212054146.
[I 2025-06-29 04:44:43,118] Trial 1 finished with value: 0.5359319189065741 and parameters: {'lr': 2.708916662683437e-05, 'n_estimators': 186, 'max_depth': 4, 'num_leaves': 63, 'subsample': 0.5372630344763973, 'colsample_bytree': 0.849442817276649, 'reg_alpha': 0.09266314028306984, 'reg_lambda': 1.1663679970572534, 'path_smooth': 2.0788347659089347e-06, 'min_data_in_leaf': 50, 'max_bin': 141, 'feature_fraction': 0.599451423457273, 'bagging_fraction': 0.7580210670788405}.

Q1 최적 파라미터: {'lr': 5.986202776859141e-05, 'n_estimators': 357, 'max_depth': 27, 'num_leaves': 19, 'subsample': 0.9742383442381624, 'colsample_bytree': 0.6345164251668162, 'reg_alpha': 0.3468796622416183, 'reg_lambda': 3.4480200794318363, 'path_smooth': 9.439199759757051e-05, 'min_data_in_leaf': 14, 'max_bin': 170, 'feature_fraction': 0.5342426870960241, 'bagging_fraction': 0.7435022912454864}
Q1 최고 F1 점수: 0.5901

=== target: Q2 ===


[I 2025-06-29 04:45:29,038] Trial 0 finished with value: 0.6120889696609465 and parameters: {'lr': 1.5106249671860432e-05, 'n_estimators': 339, 'max_depth': 31, 'num_leaves': 17, 'subsample': 0.5023240118702073, 'colsample_bytree': 0.7526598267090864, 'reg_alpha': 0.6834515648539062, 'reg_lambda': 4.603537484050716, 'path_smooth': 4.1267228833656876e-08, 'min_data_in_leaf': 74, 'max_bin': 172, 'feature_fraction': 0.6605739639515698, 'bagging_fraction': 0.592452365530872}. Best is trial 0 with value: 0.6120889696609465.
[I 2025-06-29 04:45:29,520] Trial 1 finished with value: 0.5944228664352503 and parameters: {'lr': 2.708916662683437e-05, 'n_estimators': 186, 'max_depth': 4, 'num_leaves': 63, 'subsample': 0.5372630344763973, 'colsample_bytree': 0.849442817276649, 'reg_alpha': 0.09266314028306984, 'reg_lambda': 1.1663679970572534, 'path_smooth': 2.0788347659089347e-06, 'min_data_in_leaf': 50, 'max_bin': 141, 'feature_fraction': 0.599451423457273, 'bagging_fraction': 0.7580210670788405}.

...

Q3 최적 파라미터: {'lr': 0.036898121576239715, 'n_estimators': 270, 'max_depth': 4, 'num_leaves': 41, 'subsample': 0.8029777521874408, 'colsample_bytree': 0.6991856095543015, 'reg_alpha': 0.009316649013751472, 'reg_lambda': 3.6185733461132763, 'path_smooth': 4.42523794829388e-05, 'min_data_in_leaf': 51, 'max_bin': 150, 'feature_fraction': 0.8429971019116286, 'bagging_fraction': 0.5826565656227154}
Q3 최고 F1 점수: 0.6505

=== target: S2 ===


[I 2025-06-29 04:46:33,957] Trial 0 finished with value: 0.6411782606679586 and parameters: {'lr': 1.5106249671860432e-05, 'n_estimators': 339, 'max_depth': 31, 'num_leaves': 17, 'subsample': 0.5023240118702073, 'colsample_bytree': 0.7526598267090864, 'reg_alpha': 0.6834515648539062, 'reg_lambda': 4.603537484050716, 'path_smooth': 4.1267228833656876e-08, 'min_data_in_leaf': 74, 'max_bin': 172, 'feature_fraction': 0.6605739639515698, 'bagging_fraction': 0.592452365530872}. Best is trial 0 with value: 0.6411782606679586.
[I 2025-06-29 04:46:34,673] Trial 1 finished with value: 0.6721368047943812 and parameters: {'lr': 2.708916662683437e-05, 'n_estimators': 186, 'max_depth': 4, 'num_leaves': 63, 'subsample': 0.5372630344763973, 'colsample_bytree': 0.849442817276649, 'reg_alpha': 0.09266314028306984, 'reg_lambda': 1.1663679970572534, 'path_smooth': 2.0788347659089347e-06, 'min_data_in_leaf': 50, 'max_bin': 141, 'feature_fraction': 0.599451423457273, 'bagging_fraction': 0.7580210670788405}.

S2 최적 파라미터: {'lr': 0.037532686922994406, 'n_estimators': 383, 'max_depth': 11, 'num_leaves': 45, 'subsample': 0.9588384488539574, 'colsample_bytree': 0.8662711946624343, 'reg_alpha': 0.0010098063530755898, 'reg_lambda': 1.8763937077111088, 'path_smooth': 3.66475997745807e-07, 'min_data_in_leaf': 49, 'max_bin': 248, 'feature_fraction': 0.8486662483694889, 'bagging_fraction': 0.7602864896829367}
S2 최고 F1 점수: 0.7482

=== target: S3 ===


[I 2025-06-29 04:48:00,009] Trial 0 finished with value: 0.6721487363011842 and parameters: {'lr': 1.5106249671860432e-05, 'n_estimators': 339, 'max_depth': 31, 'num_leaves': 17, 'subsample': 0.5023240118702073, 'colsample_bytree': 0.7526598267090864, 'reg_alpha': 0.6834515648539062, 'reg_lambda': 4.603537484050716, 'path_smooth': 4.1267228833656876e-08, 'min_data_in_leaf': 74, 'max_bin': 172, 'feature_fraction': 0.6605739639515698, 'bagging_fraction': 0.592452365530872}. Best is trial 0 with value: 0.6721487363011842.
[I 2025-06-29 04:48:00,710] Trial 1 finished with value: 0.6870491498389327 and parameters: {'lr': 2.708916662683437e-05, 'n_estimators': 186, 'max_depth': 4, 'num_leaves': 63, 'subsample': 0.5372630344763973, 'colsample_bytree': 0.849442817276649, 'reg_alpha': 0.09266314028306984, 'reg_lambda': 1.1663679970572534, 'path_smooth': 2.0788347659089347e-06, 'min_data_in_leaf': 50, 'max_bin': 141, 'feature_fraction': 0.599451423457273, 'bagging_fraction': 0.7580210670788405}.

S3 최적 파라미터: {'lr': 0.0003843837705965629, 'n_estimators': 215, 'max_depth': 9, 'num_leaves': 26, 'subsample': 0.8538287056229447, 'colsample_bytree': 0.8565098244976137, 'reg_alpha': 0.0018197677786855378, 'reg_lambda': 0.003125749990120914, 'path_smooth': 1.588413313101872e-06, 'min_data_in_leaf': 13, 'max_bin': 193, 'feature_fraction': 0.5202582078508149, 'bagging_fraction': 0.556077584416155}
S3 최고 F1 점수: 0.7541



In [22]:
def smote_optuna_multi(split_date):
  for col in target_multi:
    print(f'=== target: {col} ===')
    x_train, x_val, y_train, y_val = split_data[col]

    def objective_multi(trial):
      params = {
          'learning_rate': trial.suggest_float("lr", 1e-5, 1e-1, log=True),
          'n_estimators': trial.suggest_int('n_estimators', 50, 500),
          'max_depth': trial.suggest_int('max_depth', 2, 32),
          'num_leaves': trial.suggest_int('num_leaves', 16, 64),
          'subsample': trial.suggest_float('subsample', 0.5, 1.0),
          'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
          'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
          'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
          'path_smooth' : trial.suggest_loguniform('path_smooth', 1e-8, 1e-3),
          'num_leaves' : trial.suggest_int('num_leaves', 30, 200),
          'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
          'max_bin' : trial.suggest_int('max_bin', 100, 255),
          'feature_fraction' : trial.suggest_uniform('feature_fraction', 0.5, 0.9),
          'bagging_fraction' : trial.suggest_uniform('bagging_fraction', 0.5, 0.9),
          'random_state': seed,
          'n_jobs': -1,
          'verbosity': -1,
          'objective': 'multiclass',
          'num_class': 3
      }

      feat = features[col]

      clf_binary = LGBMClassifier(**params)
      pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='not majority', random_state=seed), clf_binary)

      cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
      score_macro = cross_val_score(pipeline, x_train, y_train, cv=cv, scoring='f1_macro').mean()

      trial.report(score_macro, step=0)
      if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

      return score_macro

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(objective_multi, n_trials=50)

    best_params = study.best_params
    best_score = study.best_value
    lgbm_best_param_dict[col] = best_params
    print(f"{col} 최적 파라미터: {best_params}")
    print(f"{col} 최고 F1 점수: {best_score:.4f}")
    print("=" * 50, "\n")

In [23]:
smote_optuna_multi(split_data)

[I 2025-06-29 04:49:33,643] A new study created in memory with name: no-name-d5c94a49-61a3-4193-bf5e-d747b2c83a03


=== target: S1 ===


[I 2025-06-29 04:49:42,878] Trial 0 finished with value: 0.37937091086984076 and parameters: {'lr': 1.5106249671860432e-05, 'n_estimators': 339, 'max_depth': 31, 'num_leaves': 17, 'subsample': 0.5023240118702073, 'colsample_bytree': 0.7526598267090864, 'reg_alpha': 0.6834515648539062, 'reg_lambda': 4.603537484050716, 'path_smooth': 4.1267228833656876e-08, 'min_data_in_leaf': 74, 'max_bin': 172, 'feature_fraction': 0.6605739639515698, 'bagging_fraction': 0.592452365530872}. Best is trial 0 with value: 0.37937091086984076.
[I 2025-06-29 04:49:48,644] Trial 1 finished with value: 0.3913160728828385 and parameters: {'lr': 2.708916662683437e-05, 'n_estimators': 186, 'max_depth': 4, 'num_leaves': 63, 'subsample': 0.5372630344763973, 'colsample_bytree': 0.849442817276649, 'reg_alpha': 0.09266314028306984, 'reg_lambda': 1.1663679970572534, 'path_smooth': 2.0788347659089347e-06, 'min_data_in_leaf': 50, 'max_bin': 141, 'feature_fraction': 0.599451423457273, 'bagging_fraction': 0.7580210670788405

S1 최적 파라미터: {'lr': 0.09730139534911146, 'n_estimators': 312, 'max_depth': 28, 'num_leaves': 36, 'subsample': 0.9066298636580061, 'colsample_bytree': 0.9752973267431266, 'reg_alpha': 0.003364935117313055, 'reg_lambda': 0.10835192820279202, 'path_smooth': 0.00018563312488221738, 'min_data_in_leaf': 44, 'max_bin': 235, 'feature_fraction': 0.8237316784361285, 'bagging_fraction': 0.6526288505211912}
S1 최고 F1 점수: 0.4761



In [24]:
fix_param_binary = {'random_state': seed,
                    'n_jobs': -1,
                    'verbosity': -1
                    }

fix_param_multi = {'random_sate': seed,
                   'n_jobs': -1,
                   'verbosity': -1,
                   'objective': 'multiclass',
                   'num_class': 3
                   }

In [29]:
def lgbm_smote(split_data, lgbm_best_param_dict, test_x):
  preds = {}

  for col_binary in targets_binary:
    print(col_binary)
    x_train, x_val, y_train, y_val = split_data[col_binary]
    best_param = lgbm_best_param_dict[col_binary]
    feat = features[col_binary]

    clf_binary = LGBMClassifier(**best_param, **fix_param_binary)

    smote = SMOTE(sampling_strategy='minority', random_state=seed)
    x_train_over, y_train_over = smote.fit_resample(x_train[feat], y_train)

    clf_binary.fit(
        x_train_over, y_train_over,
        eval_set=[(x_val[feat], y_val)],
        callbacks=[early_stopping(10), log_evaluation(0)],
        eval_metric='logloss'
    )

    y_pred = clf_binary.predict(x_val[feat])

    f1 = f1_score(y_val, y_pred)
    train_acc = clf_binary.score(x_train[feat], y_train)
    val_acc = clf_binary.score(x_val[feat], y_val)
    print(f'{col_binary} -> Train Acc: {train_acc:4f} | Validation Accucary: {val_acc:.4f} | Validation F1_score: {f1:.4f}')
    print()

    preds[col_binary] = clf_binary.predict(test_x[feat])

  for col_multi in target_multi:
    print(col_multi)
    x_train, x_val, y_train, y_val = split_data[col_multi]
    best_param = lgbm_best_param_dict[col_multi]
    feat = features[col_multi]

    clf_multi = LGBMClassifier(**best_param, **fix_param_multi)

    smote = SMOTE(sampling_strategy='not majority', random_state=seed)
    x_train_over, y_train_over = smote.fit_resample(x_train[feat], y_train)

    clf_multi.fit(
        x_train_over, y_train_over,
        eval_set=[(x_val[feat], y_val)],
        callbacks=[early_stopping(10), log_evaluation(0)],
        eval_metric='multi_logloss'
    )

    y_pred = clf_multi.predict(x_val[feat])

    f1 = f1_score(y_val, y_pred, average='macro')
    train_acc = clf_multi.score(x_train[feat], y_train)
    val_acc = clf_multi.score(x_val[feat], y_val)
    print(f'{col_multi} -> Train Acc: {train_acc:4f} | Validation Accucary: {val_acc:.4f} | Validation F1_score: {f1:.4f}')
    print()

    preds[col_multi] = clf_multi.predict(test_x[feat])

  return preds

In [30]:
lgbm_best_param_dict

{'Q1': {'lr': 5.986202776859141e-05,
  'n_estimators': 357,
  'max_depth': 27,
  'num_leaves': 19,
  'subsample': 0.9742383442381624,
  'colsample_bytree': 0.6345164251668162,
  'reg_alpha': 0.3468796622416183,
  'reg_lambda': 3.4480200794318363,
  'path_smooth': 9.439199759757051e-05,
  'min_data_in_leaf': 14,
  'max_bin': 170,
  'feature_fraction': 0.5342426870960241,
  'bagging_fraction': 0.7435022912454864},
 'Q2': {'lr': 0.022890774102379607,
  'n_estimators': 382,
  'max_depth': 6,
  'num_leaves': 44,
  'subsample': 0.8600967205057459,
  'colsample_bytree': 0.7330323262122243,
  'reg_alpha': 0.05235770851174116,
  'reg_lambda': 5.285835323589567,
  'path_smooth': 3.36396643632219e-05,
  'min_data_in_leaf': 77,
  'max_bin': 198,
  'feature_fraction': 0.7595587475253627,
  'bagging_fraction': 0.5266004889119342},
 'Q3': {'lr': 0.036898121576239715,
  'n_estimators': 270,
  'max_depth': 4,
  'num_leaves': 41,
  'subsample': 0.8029777521874408,
  'colsample_bytree': 0.699185609554301

In [31]:
preds_lgbm = lgbm_smote(split_data, lgbm_best_param_dict, test_x)

Q1
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[22]	valid_0's binary_logloss: 0.659594
Q1 -> Train Acc: 0.836111 | Validation Accucary: 0.5667 | Validation F1_score: 0.5895

Q2
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.693136
Q2 -> Train Acc: 0.611111 | Validation Accucary: 0.4444 | Validation F1_score: 0.3056

Q3
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[58]	valid_0's binary_logloss: 0.640403
Q3 -> Train Acc: 0.861111 | Validation Accucary: 0.5778 | Validation F1_score: 0.6667

S2
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[22]	valid_0's binary_logloss: 0.633997
S2 -> Train Acc: 0.858333 | Validation Accucary: 0.6556 | Validation F1_score: 0.7395

S3
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is

# smote + xgb

In [32]:
xgb_best_param_dict = {}

def smote_optuna_binary(split_date):
  for col in targets_binary:
    print(f'=== target: {col} ===')
    x_train, x_val, y_train, y_val = split_data[col]


    def objective_binary(trial):
      params = {
        'learning_rate': trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-3, 10.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'random_state': seed,
        'n_jobs': -1,
        'verbosity': 0
        }
      feat = features[col]
      clf_binary = XGBClassifier(**params)
      pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority', random_state=seed), clf_binary)

      cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
      score = cross_val_score(pipeline, x_train[feat], y_train, cv=cv, scoring='f1').mean()

      trial.report(score, step=0)
      if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

      return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective_binary, n_trials=50)

    best_params = study.best_params
    best_score = study.best_value
    xgb_best_param_dict[col] = best_params
    print(f"{col} 최적 파라미터: {best_params}")
    print(f"{col} 최고 F1 점수: {best_score:.4f}")
    print("=" * 50, "\n")

In [33]:
smote_optuna_binary(split_data)

[I 2025-06-29 05:18:52,029] A new study created in memory with name: no-name-d8429a5a-0b75-45f3-aad0-a7e5cc605c31


=== target: Q1 ===


[I 2025-06-29 05:18:54,955] Trial 0 finished with value: 0.5758646616541354 and parameters: {'lr': 0.00045329374539775146, 'n_estimators': 246, 'max_depth': 6, 'subsample': 0.6630368701728324, 'colsample_bytree': 0.7664427839758146, 'gamma': 0.01990134836626268, 'reg_alpha': 0.002063643198274579, 'reg_lambda': 0.08454079668095166}. Best is trial 0 with value: 0.5758646616541354.
[I 2025-06-29 05:18:55,781] Trial 1 finished with value: 0.5809137050193737 and parameters: {'lr': 2.927973447111788e-05, 'n_estimators': 129, 'max_depth': 9, 'subsample': 0.5497939434862916, 'colsample_bytree': 0.5315547847054509, 'gamma': 1.0382717649374515, 'reg_alpha': 0.9711115493393966, 'reg_lambda': 0.0037246412629940774}. Best is trial 1 with value: 0.5809137050193737.
[I 2025-06-29 05:18:56,881] Trial 2 finished with value: 0.5619856668221928 and parameters: {'lr': 0.013196565956990024, 'n_estimators': 248, 'max_depth': 4, 'subsample': 0.544533192582051, 'colsample_bytree': 0.5883162222401508, 'gamma':

Q1 최적 파라미터: {'lr': 0.0008069036895725832, 'n_estimators': 446, 'max_depth': 7, 'subsample': 0.9281988961265575, 'colsample_bytree': 0.5735701616390709, 'gamma': 0.0024659282387878272, 'reg_alpha': 9.929288074376855, 'reg_lambda': 0.06878720487034166}
Q1 최고 F1 점수: 0.6030

=== target: Q2 ===


[I 2025-06-29 05:20:22,255] Trial 0 finished with value: 0.6360365504220999 and parameters: {'lr': 0.03411841333447692, 'n_estimators': 281, 'max_depth': 4, 'subsample': 0.7610213518352231, 'colsample_bytree': 0.7238239108808857, 'gamma': 0.0012253084353959717, 'reg_alpha': 5.255674081117406, 'reg_lambda': 0.01782349169071743}. Best is trial 0 with value: 0.6360365504220999.
[I 2025-06-29 05:20:22,833] Trial 1 finished with value: 0.6034020823418415 and parameters: {'lr': 0.0009425571602059847, 'n_estimators': 78, 'max_depth': 10, 'subsample': 0.5554811698148798, 'colsample_bytree': 0.8233081631430488, 'gamma': 3.2283499965065587, 'reg_alpha': 9.2145498476379, 'reg_lambda': 0.034226879057922086}. Best is trial 0 with value: 0.6360365504220999.
[I 2025-06-29 05:20:27,967] Trial 2 finished with value: 0.6582684297410639 and parameters: {'lr': 0.008889159074463143, 'n_estimators': 400, 'max_depth': 7, 'subsample': 0.5683268150385418, 'colsample_bytree': 0.9272280310621477, 'gamma': 0.7529

...

Q3 최적 파라미터: {'lr': 0.03285475021572274, 'n_estimators': 218, 'max_depth': 10, 'subsample': 0.6378569855313776, 'colsample_bytree': 0.731721539336731, 'gamma': 0.06318409007059345, 'reg_alpha': 1.584150011413055, 'reg_lambda': 0.02320613158360135}
Q3 최고 F1 점수: 0.6574

=== target: S2 ===


[I 2025-06-29 05:24:38,176] Trial 0 finished with value: 0.7207628895673099 and parameters: {'lr': 0.0001392443885719678, 'n_estimators': 330, 'max_depth': 9, 'subsample': 0.7717492298026412, 'colsample_bytree': 0.8364806952526335, 'gamma': 0.21922856852488623, 'reg_alpha': 0.20524695834728296, 'reg_lambda': 1.7411792364300627}. Best is trial 0 with value: 0.7207628895673099.
[I 2025-06-29 05:24:48,134] Trial 1 finished with value: 0.7298797160124391 and parameters: {'lr': 0.011017800046529237, 'n_estimators': 374, 'max_depth': 4, 'subsample': 0.5148821510020193, 'colsample_bytree': 0.5792023078234702, 'gamma': 0.0037160823879131217, 'reg_alpha': 3.760507643728353, 'reg_lambda': 0.12831247928765285}. Best is trial 1 with value: 0.7298797160124391.
[I 2025-06-29 05:25:01,558] Trial 2 finished with value: 0.7088293562336888 and parameters: {'lr': 0.010175181581297904, 'n_estimators': 264, 'max_depth': 6, 'subsample': 0.941002373674396, 'colsample_bytree': 0.5988239011669034, 'gamma': 0.0

S2 최적 파라미터: {'lr': 0.008139862933617077, 'n_estimators': 158, 'max_depth': 7, 'subsample': 0.7861398389714057, 'colsample_bytree': 0.7473298055476066, 'gamma': 0.006014399146254126, 'reg_alpha': 0.05787882993508159, 'reg_lambda': 0.03303697214797071}
S2 최고 F1 점수: 0.7445

=== target: S3 ===


[I 2025-06-29 05:32:31,735] Trial 0 finished with value: 0.7456742743837882 and parameters: {'lr': 0.017723854088772514, 'n_estimators': 185, 'max_depth': 5, 'subsample': 0.9539958592674946, 'colsample_bytree': 0.9888523789876329, 'gamma': 0.00768392322070267, 'reg_alpha': 0.02494886247646693, 'reg_lambda': 0.001475078920479499}. Best is trial 0 with value: 0.7456742743837882.
[I 2025-06-29 05:33:02,118] Trial 1 finished with value: 0.7366131731978948 and parameters: {'lr': 0.0021481583221836173, 'n_estimators': 414, 'max_depth': 9, 'subsample': 0.8299700597818865, 'colsample_bytree': 0.7640526009368082, 'gamma': 0.14511083261209096, 'reg_alpha': 0.003563117650215868, 'reg_lambda': 1.5314694165333567}. Best is trial 0 with value: 0.7456742743837882.
[I 2025-06-29 05:33:07,496] Trial 2 finished with value: 0.6770981483063854 and parameters: {'lr': 1.4620669497446804e-05, 'n_estimators': 150, 'max_depth': 3, 'subsample': 0.5340590934237732, 'colsample_bytree': 0.9228801634596877, 'gamma'

S3 최적 파라미터: {'lr': 0.005605227780985878, 'n_estimators': 432, 'max_depth': 11, 'subsample': 0.5577391177025836, 'colsample_bytree': 0.8790825562440518, 'gamma': 0.02670622668572468, 'reg_alpha': 0.27861132198463695, 'reg_lambda': 0.018620914728517648}
S3 최고 F1 점수: 0.7689



In [34]:
def smote_optuna_multi(split_date):
  for col in target_multi:
    print(f'=== target: {col} ===')
    x_train, x_val, y_train, y_val = split_data[col]

    def objective_multi(trial):
      params = {'learning_rate': trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-3, 10.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'random_state': seed,
        'n_jobs': -1,
        'verbosity': 2,
        'objective': 'multiclass',
        'num_class': 3
                }
      feat = features[col]

      clf_binary = XGBClassifier(**params)
      pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='not majority', random_state=seed), clf_binary)

      cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
      score_macro = cross_val_score(pipeline, x_train, y_train, cv=cv, scoring='f1_macro').mean()

      trial.report(score_macro, step=0)
      if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

      return score_macro

    study = optuna.create_study(direction="maximize")
    study.optimize(objective_multi, n_trials=50)

    best_params = study.best_params
    best_score = study.best_value
    xgb_best_param_dict[col] = best_params
    print(f"{col} 최적 파라미터: {best_params}")
    print(f"{col} 최고 F1 점수: {best_score:.4f}")
    print("=" * 50, "\n")

In [ ]:
smote_optuna_multi(split_data)

[I 2025-06-29 05:47:42,628] A new study created in memory with name: no-name-cd2d837b-486b-4217-bc2e-b10f25499b30


=== target: S1 ===


[I 2025-06-29 05:54:50,931] Trial 0 finished with value: 0.39535728283667526 and parameters: {'lr': 0.0004955737383950412, 'n_estimators': 463, 'max_depth': 8, 'subsample': 0.7767024866950202, 'colsample_bytree': 0.9592558366686643, 'gamma': 0.04342058961280567, 'reg_alpha': 0.5151986730438072, 'reg_lambda': 8.498769313357027}. Best is trial 0 with value: 0.39535728283667526.
[I 2025-06-29 05:56:05,303] Trial 1 finished with value: 0.39896095053188596 and parameters: {'lr': 0.0013046489603367158, 'n_estimators': 358, 'max_depth': 5, 'subsample': 0.7948366045763231, 'colsample_bytree': 0.6086574267061724, 'gamma': 5.983301493017975, 'reg_alpha': 0.612118782233696, 'reg_lambda': 0.0035063270466254817}. Best is trial 1 with value: 0.39896095053188596.
[I 2025-06-29 05:56:31,483] Trial 2 finished with value: 0.4158889163952635 and parameters: {'lr': 0.0034401271612133286, 'n_estimators': 195, 'max_depth': 3, 'subsample': 0.5497331764440694, 'colsample_bytree': 0.533049824295162, 'gamma': 3

In [ ]:
fix_param_binary = {'random_state': seed,
                    'n_jobs': -1,
                    'verbosity': -1
                    }

fix_param_multi = {'random_sate': seed,
                   'n_jobs': -1,
                   'verbosity': -1,
                   'objective': 'multiclass',
                   'num_class': 3
                   }

In [ ]:
def xgb_smote(split_data, xgb_best_param_dict, test_x):
  preds = {}

  for col_binary in targets_binary:
    print(col_binary)
    x_train, x_val, y_train, y_val = split_data[col_binary]
    best_param = xgb_best_param_dict[col_binary]
    feat = features[col_binary]

    clf_binary = XGBClassifier(**best_param, **fix_param_binary,
                               use_label_encoder=False
    )

    smote = SMOTE(sampling_strategy='minority', random_state=seed)
    x_train_over, y_train_over = smote.fit_resample(x_train[feat], y_train)

    clf_binary.fit(
        x_train_over, y_train_over,
        eval_set=[(x_val[feat], y_val)],
        early_stopping_rounds=10,
        verbose=False,
        eval_metric='logloss'
    )

    y_pred = clf_binary.predict(x_val[feat])

    f1 = f1_score(y_val, y_pred)
    train_acc = clf_binary.score(x_train[feat], y_train)
    val_acc = clf_binary.score(x_val[feat], y_val)
    print(f'{col_binary} -> Train Acc: {train_acc:4f} | Validation Accucary: {val_acc:.4f} | Validation F1_score: {f1:.4f}')
    print()

    preds[col_binary] = clf_binary.predict(test_x[feat])

  for col_multi in target_multi:
    print(col_multi)
    x_train, x_val, y_train, y_val = split_data[col_multi]
    best_param = xgb_best_param_dict[col_multi]
    feat = features[col_multi]

    clf_multi = XGBClassifier(**best_param, **fix_param_multi,
                              use_label_encoder=False
    )

    smote = SMOTE(sampling_strategy='not majority', random_state=seed)
    x_train_over, y_train_over = smote.fit_resample(x_train[feat], y_train)

    clf_multi.fit(
        x_train_over, y_train_over,
        eval_set=[(x_val[feat], y_val)],
        early_stopping_rounds=10,
        verbose=False,
        eval_metric='mlogloss'
    )

    y_pred = clf_multi.predict(x_val[feat])

    f1 = f1_score(y_val, y_pred, average='macro')
    train_acc = clf_multi.score(x_train[feat], y_train)
    val_acc = clf_multi.score(x_val[feat], y_val)
    print(f'{col_multi} -> Train Acc: {train_acc:4f} | Validation Accucary: {val_acc:.4f} | Validation F1_score: {f1:.4f}')
    print()

    preds[col_multi] = clf_multi.predict(test_x[feat])

  return preds

In [ ]:
xgb_best_param_dict

In [ ]:
preds = xgb_smote(split_data, xgb_best_param_dict, test_x)

# smote + catb

In [ ]:
catb_best_param_dict = {}

def smote_optuna_binary(split_date):
  for col in targets_binary:
    print(f'=== target: {col} ===')
    x_train, x_val, y_train, y_val = split_data[col]


    def objective_binary(trial):
      params = {'iterations' :trial.suggest_int("iterations", 100, 1000),
        'learning_rate': trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
        'depth': trial.suggest_int("depth", 4, 10),
        'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1e-3, 100.0, log=True),
        'bootstrap_type': trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        'random_strength': trial.suggest_float("random_strength", 1e-3, 10.0, log=True),
        'bagging_temperature': trial.suggest_float("bagging_temperature", 0.0, 10.0),
        'od_type': trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        'od_wait': trial.suggest_int("od_wait", 10, 50),
        'border_count': trial.suggest_int("border_count", 32, 255),
        'random_state': seed,
        'verbose': 0
                }

      feat = features[col]
      clf_binary = CatBoostClassifier(**params)
      pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority', random_state=seed), clf_binary)

      cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
      score = cross_val_score(pipeline, x_train[feat], y_train, cv=cv, scoring='f1').mean()

      trial.report(score, step=0)
      if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

      return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective_binary, n_trials=50)

    best_params = study.best_params
    best_score = study.best_value
    catb_best_param_dict[col] = best_params
    print(f"{col} 최적 파라미터: {best_params}")
    print(f"{col} 최고 F1 점수: {best_score:.4f}")
    print("=" * 50, "\n")

In [ ]:
smote_optuna_binary(split_data)

In [ ]:
def smote_optuna_multi(split_date):
  for col in target_multi:
    print(f'=== target: {col} ===')
    x_train, x_val, y_train, y_val = split_data[col]

    def objective_multi(trial):
      params = {'learning_rate': trial.suggest_float("lr", 1e-5, 1e-1, log=True),
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'random_state': seed,
                'verbose': 0,
                'loss_function': 'MultiClass'
                }
      feat = features[col]

      clf_binary = CatBoostClassifier(**params)
      pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='not majority', random_state=seed), clf_binary)

      cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
      score_macro = cross_val_score(pipeline, x_train, y_train, cv=cv, scoring='f1_macro').mean()

      trial.report(score_macro, step=0)
      if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

      return score_macro

    study = optuna.create_study(direction="maximize")
    study.optimize(objective_multi, n_trials=50)

    best_params = study.best_params
    best_score = study.best_value
    catb_best_param_dict[col] = best_params
    print(f"{col} 최적 파라미터: {best_params}")
    print(f"{col} 최고 F1 점수: {best_score:.4f}")
    print("=" * 50, "\n")

In [ ]:
smote_optuna_multi(split_data)

In [ ]:
fix_param_binary = {'random_state': seed,
                    'n_jobs': -1,
                    'verbose': False
                    }

fix_param_multi = {'random_sate': seed,
                   'n_jobs': -1,
                   'verbose': False,
                   'loss_function': 'MultiClass'
                   }

In [ ]:
def catb_smote(split_data, xgb_best_param_dict, test_x):
  preds = {}

  for col_binary in targets_binary:
    print(col_binary)
    x_train, x_val, y_train, y_val = split_data[col_binary]
    best_param = catb_best_param_dict[col_binary]
    feat = features[col_binary]

    clf_binary = CatBoostClassifier(**best_param, **fix_param_binary)

    smote = SMOTE(sampling_strategy='minority', random_state=seed)
    x_train_over, y_train_over = smote.fit_resample(x_train[feat], y_train)

    clf_binary.fit(
        x_train_over, y_train_over,
        eval_set=(x_val[feat], y_val),
        early_stopping_rounds=10,
        verbose=False,
        eval_metric='Logloss'
        )

    y_pred = clf_binary.predict(x_val[feat])

    f1 = f1_score(y_val, y_pred)
    train_acc = clf_binary.score(x_train[feat], y_train)
    val_acc = clf_binary.score(x_val[feat], y_val)
    print(f'{col_binary} -> Train Acc: {train_acc:4f} | Validation Accucary: {val_acc:.4f} | Validation F1_score: {f1:.4f}')
    print()

    preds[col_binary] = clf_binary.predict(test_x[feat])

  for col_multi in target_multi:
    print(col_multi)
    x_train, x_val, y_train, y_val = split_data[col_multi]
    best_param = catb_best_param_dict[col_multi]
    feat = features[col_multi]

    clf_multi = CatBoostClassifier(**best_param, **fix_param_multi)

    smote = SMOTE(sampling_strategy='not majority', random_state=seed)
    x_train_over, y_train_over = smote.fit_resample(x_train[feat], y_train)

    clf_multi.fit(
        x_train_over, y_train_over
        eval_set=(x_val[feat], y_val),
        early_stopping_rounds=10,
        verbose=False,
        eval_metric='MultiClass'
    )

    y_pred = clf_multi.predict(x_val[feat])

    f1 = f1_score(y_val, y_pred, average='macro')
    train_acc = clf_multi.score(x_train[feat], y_train)
    val_acc = clf_multi.score(x_val[feat], y_val)
    print(f'{col_multi} -> Train Acc: {train_acc:4f} | Validation Accucary: {val_acc:.4f} | Validation F1_score: {f1:.4f}')
    print()

    preds[col_multi] = clf_multi.predict(test_x[feat])

  return preds

In [ ]:
catb_best_param_dict

In [ ]:
preds = catb_smote(split_data, catb_best_param_dict, test_x)

# voting

In [ ]:
def prediction(models, test, mode=None, weights=None):
    #hard voting일 경우
    if mode == "hard":
        preds = np.asarray([x.predict(test).reshape(-1) for x in models]).T
        res = np.apply_along_axis(
            lambda x: np.argmax(np.bincount(x, weights=weights)),
            axis=1,
            arr=preds
        )
    #soft voting일 경우
    elif mode == "soft":
        preds = np.asarray([x.predict_proba(test) for x in models])
        res = np.zeros(preds[0].shape)
        for pred, weight in zip(preds, weights):
            res = res + pred*weight
        res = np.argmax(preds, axis=0)
    else:
        res = models[0].predict(test)
    return res

#hard voting을 선택하여 모델 중요도별 가중치 제공.
preds = prediction(models, test_x, 'hard', [2,5,4] )

In [ ]:
def voting(split_data,
           lgbm_best_param_dict, xgb_best_param_dict, catb_best_param_dict,
           test_x):

    preds = {}

    # 이진 분류
    for col_binary in targets_binary:
        x_train, x_val, y_train, y_val = split_data[col_binary]
        feat = features[col_binary]

        lgbm = LGBMClassifier(**lgbm_best_param_dict[col_binary],
                              random_state = seed,
                              n_jobs = -1,
                              verbosity = -1)
        xgb = XGBClassifier(**xgb_best_param_dict[col_binary],
                            random_state = seed,
                            n_jobs = -1,
                            verbosity = 0)
        catb = CatBoostClassifier(**catb_best_param_dict[col_binary],
                                  random_state = seed,
                                  n_jobs=-1,
                                  verbose=False)

        smote = SMOTE(sampling_strategy='minority', random_state=seed)
        x_train_over, y_train_over = smote.fit_resample(x_train[feat], y_train)

        voting_binary = VotingClassifier(
            estimators=[('lgbm', lgbm), ('xgb', xgb), ('catb', catb)],
            voting='hard'
        )

        voting_binary.fit(x_train_over, y_train_over)
        y_pred = voting_binary.predict(x_val[feat])

        f1 = f1_score(y_val, y_pred)
        train_acc = voting_binary.score(x_train[feat], y_train)
        val_acc = voting_binary.score(x_val[feat], y_val)

        print(f'Binary {col_binary} -> Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val F1: {f1:.4f}')
        preds[col_binary] = voting_binary.predict(test_x[feat])

    # 다중 분류
    for col_multi in target_multi:
        x_train, x_val, y_train, y_val = split_data[col_multi]
        feat = features[col_multi]

        model1 = LGBMClassifier(**lgbm_param[col_multi],
                                random_sate = seed,
                                n_jobs =-1,
                                verbosity = -1,
                                objective = 'multiclass',
                                num_class = 3
        model2 = XGBClassifier(**xgb_param[col_multi],
                               random_sate = seed,
                               n_jobs =-1,
                               verbosity = 0,
                               objective = 'multiclass',
                               num_class = 3)
        model3 = CatBoostClassifier(**catb_param,
                                    random_sate = seed, n_jobs = -1,
                                    verbose = False,
                                    loss_function = 'MultiClass')

        smote = SMOTE(sampling_strategy='not majority', random_state=seed)
        x_train_over, y_train_over = smote.fit_resample(x_train[feat], y_train)

        voting_multi = VotingClassifier(
            estimators=[('lgbm', model1), ('xgb', model2), ('catb', model3)],
            voting='soft'
        )

        voting_multi.fit(x_train_over, y_train_over)
        y_pred = voting_multi.predict(x_val[feat])

        f1 = f1_score(y_val, y_pred, average='macro')
        train_acc = voting_multi.score(x_train[feat], y_train)
        val_acc = voting_multi.score(x_val[feat], y_val)

        print(f'Multi {col_multi} -> Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val F1: {f1:.4f}')
        preds[col_multi] = voting_multi.predict(test_x[feat])

    return preds

In [ ]:
preds = voting(split_data, lgbm_best_param_dict, xgb_best_param_dict, catb_best_param_dict, test_x)

# stacking

# 결과저장

In [ ]:
final = sample_submission[['subject_id','sleep_date','lifelog_date']].copy()
final['lifelog_date'] = pd.to_datetime(final['lifelog_date']).dt.date

final['id'] = final['subject_id'] + '_' + final['lifelog_date'].astype(str)
assert len(final) == len(preds['S1'])

target_list = ['Q1', 'Q2', 'Q3', 'S1','S2','S3']
for col in target_list:
  final[col] = preds[col].astype(int)

submission_final = final[['subject_id','sleep_date','lifelog_date', 'Q1','Q2','Q3','S1','S2','S3']]
submission_final.to_csv('submission_final.csv',index=False)

from google.colab import files
files.download("submission_final.csv")